In [17]:
!git clone https://github.com/antonzub99/video-coloring.git

Cloning into 'video-coloring'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 71 (delta 27), reused 56 (delta 16), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [18]:
!mv video-coloring/* .

# Frame-wize colorization without temporary consistency

## Data preparation

We select 5 last videos fram DAVIS dataset and a fragment from Andrey Rublev black-and-white film.

In [19]:
!wget https://data.vision.ee.ethz.ch/csergi/share/davis/DAVIS-2017-trainval-480p.zip
!unzip -d ./videos ./DAVIS-2017-trainval-480p.zip

--2022-05-26 12:41:05--  https://data.vision.ee.ethz.ch/csergi/share/davis/DAVIS-2017-trainval-480p.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 832766765 (794M) [application/zip]
Saving to: ‘DAVIS-2017-trainval-480p.zip.1’

DAVIS-2017-trainval 100%[===================>] 794.19M  14.5MB/s    in 49s     

2022-05-26 12:41:54 (16.3 MB/s) - ‘DAVIS-2017-trainval-480p.zip.1’ saved [832766765/832766765]

Archive:  ./DAVIS-2017-trainval-480p.zip
replace ./videos/DAVIS/Annotations/480p/bear/00000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [20]:
!gdown 1UObl9IPKHLZcKSxK7Lhl4QGmxra6rbLg

Downloading...
From: https://drive.google.com/uc?id=1UObl9IPKHLZcKSxK7Lhl4QGmxra6rbLg
To: /content/val.zip
100% 5.59M/5.59M [00:00<00:00, 193MB/s]


In [21]:
!unzip -d ./rublev val.zip

Archive:  val.zip
replace ./rublev/img001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [25]:
import dataset 

In [26]:
dawis_dataset = dataset.VideoDataset('./videos/DAVIS/JPEGImages/480p', frame_stack=1, img_size=128, need_full_videos=True)

100%|██████████| 90/90 [01:43<00:00,  1.15s/it]


In [27]:
rublev = dataset.OneBWVideoDataset('./rublev', frame_stack=5, img_size=128)

In [28]:
from torch.utils.data import DataLoader

rublev_dataset = DataLoader(rublev, num_workers=2, batch_size=32, shuffle=False)

### Run CIC network for DAVIS testing part

In [33]:
%load_ext autoreload
%autoreload 2

In [34]:
import framewise

In [37]:
model = framewise.eccv16_framewise(pretrained=True).eval().cuda()

[autoreload of framewise failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 0 free vars, not 1
]


In [40]:
import matplotlib.pyplot as plt
import imageio
import cv2
import torch.nn.functional as F
import torch
import numpy as np
from skimage import color

for i in range(85, 90):
    with torch.no_grad():
        # C x T x H x W
        lum, ab = dawis_dataset[i]
        # C x T x H x W
        pred_ab = model(lum[None,...].float().cuda()).cpu()[0]

        lab_preds = torch.cat((lum, pred_ab), dim=0).permute(1,2,3,0).numpy()
        lab_reals = torch.cat((lum, ab), dim=0).permute(1,2,3,0).numpy()

        img_pred = torch.tensor(color.lab2rgb(lab_preds))
        img_real =  torch.tensor(color.lab2rgb(lab_reals))

        video = np.concatenate((lum.repeat(3,1,1,1).permute(1,2,3,0) / 100, img_pred, img_real), axis=2)
    

    gif_name = 'DAVIS_{}.gif'.format(i)
    imageio.mimsave(gif_name, (np.clip(video, 0, 1) * 255).astype(np.uint8))

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


Final videos:

<img src="https://imgur.com/bQLBK3N.gif">


<img src="https://imgur.com/OtHOZ3v.gif">


<img src="https://imgur.com/3nrR46W.gif">


<img src="https://imgur.com/w5uRFNG.gif">


<img src="https://imgur.com/IBFkUQq.gif">


### Run on RublevNet dataset

In [41]:
rublev_video = []

with torch.no_grad():
    for batch in rublev_dataset:
        lum = batch.float().cuda()
        ab = model(lum).cpu()

        lab = torch.cat((lum.cpu(), ab), dim=1).permute(0,2,3,4,1).flatten(end_dim=1)
        rgb = color.lab2rgb(lab)
        wb = lum.cpu().repeat(1,3,1,1,1).permute(0,2,3,4,1).flatten(end_dim=1).numpy()

        rublev_video.append(np.concatenate((wb / 100, rgb), axis=2))


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 106 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 4507 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [42]:
final_rublev = np.concatenate(rublev_video, axis=0)

In [43]:
final_rublev.shape

(375, 128, 256, 3)

In [44]:
imageio.mimsave('rublev.gif', (np.clip(final_rublev, 0, 1) * 255).astype(np.uint8))

Result:

<img width=512 src="https://imgur.com/Xxko6NI.gif">